In [1]:
import pandas as pd

D:\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
import re
def  clean_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df
test_clean = clean_text(test, "tweet")
train_clean = clean_text(train, "tweet")

In [9]:
#Resample to handle imbalanced dataset

In [8]:
from sklearn.utils import resample
train_majority = train_clean[train_clean.label==0]
train_minority = train_clean[train_clean.label==1]
train_minority_upsampled = resample(train_minority, 
                                 replace=True,    
                                 n_samples=len(train_majority),   
                                 random_state=123)
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['label'].value_counts()

1    29720
0    29720
Name: label, dtype: int64

In [10]:
import tensorflow as tf
text_vectorizer = tf.keras.layers.TextVectorization(
     output_mode='int', pad_to_max_tokens=True,max_tokens=2500 , standardize='lower_and_strip_punctuation')
text_vectorizer.adapt(train_upsampled['tweet'])

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input,Embedding

In [12]:
model = Sequential([
    Input((1,),dtype=tf.string),
    text_vectorizer,
    Embedding(2500,8),
    LSTM(64, activation='relu'),
    Dense(1,activation='sigmoid')
]
)

In [13]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics='accuracy')

In [ ]:
model.fit(train_upsampled['tweet'], train_upsampled['label'], epochs=10)

In [16]:
model.predict(test_clean['tweet'])

array([[1.2897766e-01],
       [6.4211261e-01],
       [4.2913254e-07],
       ...,
       [1.2341881e-01],
       [4.1490424e-07],
       [4.3638319e-02]], dtype=float32)